In [270]:
import sqlite3, random, pathlib
from search import search_library
from dotenv import load_dotenv
from openai import OpenAI, OpenAIError
from pydantic import BaseModel, Field, ValidationError
import os
from openai import OpenAI
from pydantic import BaseModel
import json
import time
import random
from helpers.sentence_splitter import split_sentences
from time import sleep
from tqdm import tqdm  # optional: pip install tqdm

def ask(sys_msg, usr_msg):
    resp = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": usr_msg}
        ]
    )
    return resp.choices[0].message.content

def ask_func(sys_msg, usr_msg, FUNC):
    resp = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[{"role":"system","content":sys_msg}, {"role":"user","content":usr_msg}],
            tools=[FUNC], tool_choice="auto", temperature=0.5
            )
    return resp


def print_object_fields(obj: dict, index: int = 1, fields: list[str] = ["name", "text"]) -> None:
    print(f"{index} -")
    for field in fields:
        print(f"{field}: {obj.get(field, '[Missing]')}")


def parse_function_response(
    response,
    obj_key: str = "element",
    fields: list[str] = ["name", "text"],
    display: bool = True
) -> list[dict]:
    """Parse tool call response and return a list of objects with optional printing."""
    args_raw = response.choices[0].message.tool_calls[0].function.arguments
    objects = json.loads(args_raw).get(obj_key, [])

    result = []
    for i, obj in enumerate(objects, 1):
        if display:
            print_object_fields(obj, index=i, fields=fields)
        result.append(obj)

    return result

def process_response(response, sys_msg=SYS_PROMPT_PARSE_ELEMENTS, ELEM_FUNC=ELEM_FUNC):
    responses_func = ask_func(sys_msg, response, ELEM_FUNC)
    responses_raw = parse_function_response(responses_func, display=False)
    responses = []
    for response_raw in responses_raw:
        n, t = response_raw['name'], response_raw['text']
        responses.append({n: t})
    return responses


load_dotenv()
client = OpenAI()
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")

DB_FILE = "library_temp.sqlite"
MAX_RESULTS = 100


SYS_PROMPT_PARSE_ELEMENTS = ("Read the following text and separate each element of the suggested story framework into a object with the name and text.\n"
        "You should return a list of objects containing each story element in a given structure.\n"
        "Do not add or remove text; your job is to parse and deconstruct the text string into the given structure.\n"
        "Treat every line that begins with a number followed by “.” and a space (e.g. “1. ”) or **Element** as the NAME of a new element."
        "Capture all following lines up to—but not including—the next such line as the TEXT for that element.")

SYS_PROMPT_PARSE_ELEMENTS_MODIFIED = ("Read the following text and separate each element of the suggested story framework into a object with the name and text.\n"
        "You should return a list of objects containing each story element in a given structure.\n"
        "Do not add or remove text; your job is to parse and deconstruct the text string into the given structure.\n"
        "Treat every line that begins with 'Element:' followed by a word/phrase as the NAME of a new element."
        "Capture all following lines up to—but not including—the next such line as the TEXT for that element.")

SYSTEM_PROMPT_IMAGINE = """
Your task is to take a human imaginative directive and translate it into a coherent, structured list of domains, technologies, and disciplines that could be explored to realize or approximate that imaginative directive.
The goal is to break down the directive into actionable or researchable avenues across science, technology, psychology, education, or other relevant fields.
Each entry should be self-contained, meaning it should be understandable on its own, and clearly tied back to the core imaginative directive.
"""


ELEM_FUNC = {
    "type": "function",
    "function": {
        "name": "element_function",
        "description": "Read the following text and separate each element of the suggested story framework into a object with the name and text. \
        You should return a list of objects containing each story element in a given structure. \
        Do not add or remove text; your job is to parse and deconstruct the text string into the given structure.",
        "parameters": {
            "type": "object",
            "properties": {
                "element": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": { "type": "string" },
                            "text": { "type": "string" }
                        },
                        "required": ["name", "text"]
                    }
                }
            },
            "required": ["element"]
        }
    }
}




In [ ]:
imagination = {}
story_space = {}

In [381]:
story_space = {}

In [379]:
fantasia = "I imagine creating a Youtube channel."

sys_msg = SYSTEM_PROMPT_IMAGINE
usr_msg = fantasia

seeds_resp = ask(sys_msg, usr_msg)
print(seeds_resp)

sys_msg = SYS_PROMPT_PARSE_ELEMENTS
seeds_func = ask_func(sys_msg, seeds_resp, ELEM_FUNC)
seeds_raw = parse_function_response(seeds_func, display=False)
seeds = []
for seed_raw in seeds_raw:
    n, t = seed_raw['name'], seed_raw['text']
    seeds.append({n: t})
    
imagination[fantasia] = seeds


To realize the imaginative directive of creating a YouTube channel, various domains, technologies, and disciplines can be explored:

1. **Content Creation and Storytelling**
   - **Scriptwriting:** Develop compelling scripts to keep viewers engaged.
   - **Video Production:** Master video shooting techniques, lighting, and framing.
   - **Editing Skills:** Learn to use editing software such as Adobe Premiere Pro or Final Cut Pro.

2. **Digital Marketing and SEO**
   - **Search Engine Optimization (SEO):** Utilize SEO strategies to increase video discoverability on YouTube.
   - **Social Media Marketing:** Use platforms like Instagram, Twitter, and Facebook to promote content and engage with the audience.
   - **Audience Analytics:** Study YouTube analytics for targeting content and growing subscribers.

3. **Technology and Software**
   - **Camera Equipment:** Research and invest in suitable cameras and audio equipment.
   - **Editing Software:** Understand various editing software opt

In [380]:
for fantasia, dimensions in list(imagination.items())[1:2]:
    print(fantasia)
    for dimension in dimensions[1:2]:
        dim_dir, dim_text = list(dimensions.items())[0]
        print(seed_dir)
        print(seed_text, "\n\n")

        # what are the dimensions of story-telling along this dimension

        # loop dimensions and see what happens?
        element_text = direction_to_string(element)
        sys_msg = SYSTEM_PROMPT_CONSTRUCTION_DIMENSIONS
        usr_msg = USER_PROMPT_CONSTRUCTION_DIMENSIONS.format(fantasia=fantasia, direction_text=direction_to_string(direction))
        dimensions_resp = ask(sys_msg, usr_msg)
        print(dimensions_resp)
        # what are the sources of information? 

        #sys_msg = SYSTEM_PROMPT_GENERALIZED_ELEMENTS
        #usr_msg = USER_PROMPT_GENERALIZED_ELEMENTS.format(fantasia=fantasia, seed_direction=seed_dir, seed_text=seed_text)
        #elements_resp = ask(sys_msg, usr_msg)
        #print(elements_resp, "\n\n\n")
        sys_msg = SYS_PROMPT_PARSE_ELEMENTS
        elements_func = ask_func(sys_msg, elements_resp, ELEM_FUNC)
        elements_raw = parse_function_response(elements_func, display=False)
        elements = []
        for element_raw in elements_raw:
            n, t = element_raw['name'], element_raw['text']
            print(f"{n}\n{t}\n\n")
            elements.append({n: t})
            
        environment.setdefault(fantasia, []).append(elements)
        
        



I dream of burgers.
2. Food Science and Technology
- **Flavor Profiling**: Utilize analytical chemistry to identify and replicate the ideal flavor profiles in burger components.
- **Texture Analysis**: Use rheology and material science to create patties with desirable mouth-feel and structural integrity.
- **Food Preservation**: Investigate methods to extend shelf life of burger ingredients without compromising taste or quality. 


Fantasia: "I dream of burgers."

2. Food Science and Technology
- **Flavor Synthesis Matrix**: A comprehensive database that maps out the chemical compounds responsible for diverse burger flavors, allowing for precise flavor engineering.
- **Gastro-Rheological Simulator**: A computational tool that models the textural dynamics of burger components during chewing, helping to design patties with optimal mouth-feel.
- **Biodegradable Encapsulation**: Microencapsulation technology for essential flavors and nutrients, promoting longer shelf life while maintaining

In [217]:
print(SYSTEM_PROMPT_GENERALIZED_ELEMENTS)


You are a multi-domain synthesis engine designed to interpret human fantasias—wishes, dreams, or creative intentions—and translate them into actionable or imaginative elements across diverse fields of knowledge.

Given:
- A fantasia (e.g., "I wish I could learn instantly" or "I dream of burgers")
- A set of directions (scientific, technological, cultural, artistic, speculative, etc.)

Your task is to analyze each direction and generate **a structured list of storytelling or design elements** that help explore, support, or realize the fantasia.

Each element should:
- Be clearly and concisely named
- Be understandable on its own (self-contained)
- Represent a concept, process, structure, behavior, interaction, or constraint relevant to the fantasia
- Be usable as part of a narrative, system model, speculative fiction, product concept, or philosophical exploration

You may interpret the fantasia literally, metaphorically, or speculatively—but always ensure the connection is meaningful a

In [215]:
print(len(elements))
print(elements[1])

2
{'1. Neuroscience Direction': "- **Brain-Computer Interface (BCI) Adaptation**: Explore a system enabling direct neural communication with digital devices, allowing rapid information transfer to the brain.\n- **Synaptic Acceleration Protocols**: Develop protocols that enhance synaptic connections, speeding up the learning process by mimicking accelerated memory formation.\n- **Neurochemical Modulators**: Design compounds or devices that optimize neurotransmitter levels to enhance cognitive functions linked to rapid learning and memory retention.\n- **Neuroplasticity Enhancement Techniques**: Create techniques, such as targeted electrical stimulation, that maximize the brain's natural ability to reorganize and adapt when learning new topics.\n- **Memory Fusion Algorithms**: Develop algorithms that enable cohesive integration of new information into existing neural networks for seamless knowledge acquisition.\n- **Cognitive Load Management Systems**: Innovate systems that dynamically a

In [205]:
SYSTEM_PROMPT_ELEMENT_TO_DIMENSIONS = """
You are a story systems architect.

Given a narrative or conceptual element, your task is to identify the **set of dimensions** required to tell a complete, coherent story involving that element. These dimensions define the structure, context, and transformational arc of the element in a world or system.

For the given element, list the core dimensions and provide a short description for each.

You may draw from narrative theory, systems design, speculative fiction, mythology, game design, or cognitive science.

Each dimension should help the user *instantiate* the element in a functional or narrative context.

Output Format:

Element: [element name]

Required Dimensions:
1. [Dimension Name]: [Description]
2. [Dimension Name]: [Description]
...
"""


In [232]:
SYSTEM_PROMPT_FANTASIA_TO_ELEMENTS = """
You are a multi-domain imagination engine.

Given a human fantasia (wish, dream, or aspiration) and a set of directions (fields of study, technologies, disciplines, or creative frameworks), your task is to generate **narrative or conceptual elements** that could be used to explore, design, or realize the fantasia.

For each direction, do the following:
- Interpret its subtopics in the context of the fantasia.
- Invent a set of imaginative but coherent elements (tools, systems, processes, interfaces, environments, entities, rituals, or principles).
- Name each element clearly.
- Make each element self-contained: include a brief explanation so it can stand alone as a modular building block.
- Draw freely from speculative science, psychology, design fiction, metaphors, myth, or emerging technologies—but maintain internal logic and grounding in the subtopic.

Output Format:

1. [Direction Name]
- **[Element Name]**: [Concise, self-contained explanation]
- **[Element Name]**: [Explanation]
...
(Repeat for each direction)

Do not cite sources. Do not summarize. Instead, generate **creative design primitives or storytelling ingredients** that could populate worlds, systems, or frameworks centered on the fantasia.
"""

USER_PROMPT_FANTASIA_TO_ELEMENTS = """
Fantasia: "{fantasia}"
Direction: {seed_direction}
Subtopics:
{seeds_text}

List the best system/design elements to explore these topics.
"""

In [228]:

print(len(seeds))
print(seeds)
seeds_text = ""
for seed in seeds:
    direction, text = list(seed.items())[0]
    seeds_text += f"{direction}\n{text}\n\n"
print(seeds_text)
    

8
[{'1. Neuroscience': '- **Neural Plasticity Research**: Investigate how the brain can be trained or enhanced to assimilate information more quickly.\n- **Memory Encoding**: Study the mechanisms behind how memories are encoded, stored, and retrieved to improve learning speed.'}, {'2. Artificial Intelligence and Machine Learning': '- **Natural Language Processing (NLP)**: Develop AI systems that can effectively summarize and explain complex topics instantly.\n- **Reinforcement Learning Models**: Apply AI that mimics human learning processes, but at an accelerated rate, to train individuals rapidly through adaptive learning platforms.'}, {'3. Cognitive Psychology': '- **Rapid Skill Acquisition**: Explore techniques and strategies for speeding up the learning process through optimized cognitive practices.\n- **Attention and Focus Training**: Identify methods or technologies that enhance concentration, crucial for absorbing new information quickly.'}, {'4. Educational Technology': '- **Ad

In [233]:
sys_msg = SYSTEM_PROMPT_FANTASIA_TO_ELEMENTS
usr_msg = USER_PROMPT_FANTASIA_TO_ELEMENTS.format(fantasia=fantasia, seed_direction=seed_dir, seeds_text=seeds_text)
elements_resp = ask(sys_msg, usr_msg)
print(elements_resp)


1. Neuroscience
- **NeuroCortex Sync (NCS)**: A speculative device that harnesses enhanced neural plasticity to synchronize brainwave patterns across individuals and devices, allowing rapid uptake of new information.
- **Memory Imprint Chronograph (MIC)**: An advanced mechanism that creates a 'timestamp' in the brain's memory storage regions, allowing faster retrieval and solidifying newly acquired knowledge.

2. Artificial Intelligence and Machine Learning
- **Synapse AI Tutor**: An AI-based personal tutor that uses natural language processing to convert complex topics into simple, digestible summaries specifically tailored to the learner's context.
- **AccelEdu RL Engine**: A system utilizing reinforcement learning to tailor learning experiences by simulating challenges and rewards, ensuring efficient knowledge acquisition and retention.

3. Cognitive Psychology
- **NeuronVibe Pulse Timer**: A small wearable that sends rhythmic pulses to the wearer, synchronizing with cognitive rhyt

In [297]:
SYSTEM_PROMPT_CONSTRUCTION_DIMENSIONS = """
You are a systems design analyst.

Your task is to analyze a list of conceptual or functional elements derived from a human fantasia (wish, dream, or aspiration). For each element, describe its **Mechanism of Action** dimension as it relates to telling a complete narrative of how the element works.

For each element, explain:
1. **Mechanism of Action – Description**  
   - How the element operates internally to achieve its function.
   - Focus on the causal transformations, internal logic, and dynamic interactions involved.

2. **Mechanism of Action – Key Questions**  
   - List the most important questions that must be answered to fully understand or build the mechanism of this element.

3. **Mechanism of Action – Required Information**  
   - Describe the kinds of models, inputs, or system-level information needed to construct or simulate the mechanism of action for this element.

You may reference theories, biological systems, machines, learning models, or imagined technologies if relevant.

Be specific. Use the element definition to ground your analysis. Do not repeat generic definitions. Focus on telling a complete functional narrative **for each element individually**.
"""

USER_PROMPT_CONSTRUCTION_DIMENSIONS = """
Fantasia: "{fantasia}"

The following is a list of elements under a given domain.

{direction_text}

For each element, list the core dimensions along the mechanism of action (with a description) which are required to tell a complete functional narrative of this element.
"""



In [250]:
directions = process_response(elements_resp)

In [247]:
element = elements[0]
def direction_to_string(element):
    n, t = list(element.items())[0]
    return f"{n}\n{t}"

In [296]:
print(usr_msg)


Fantasia: "I wish that I could learn any topic instantly."

The following is a list of elements under a given domain.

1. Neuroscience
- **NeuroCortex Sync (NCS)**: A speculative device that harnesses enhanced neural plasticity to synchronize brainwave patterns across individuals and devices, allowing rapid uptake of new information.
- **Memory Imprint Chronograph (MIC)**: An advanced mechanism that creates a 'timestamp' in the brain's memory storage regions, allowing faster retrieval and solidifying newly acquired knowledge.

For each element, list the core dimensions along the mechanism of action (with a description) which are required to tell a complete functional narrative of this element.



In [298]:
for direction in directions[0:1]:
    element_text = direction_to_string(element)
    sys_msg = SYSTEM_PROMPT_CONSTRUCTION_DIMENSIONS
    usr_msg = USER_PROMPT_CONSTRUCTION_DIMENSIONS.format(fantasia=fantasia, direction_text=direction_to_string(direction))
    dimensions_resp = ask(sys_msg, usr_msg)
    print(dimensions_resp)

### Element 1: NeuroCortex Sync (NCS)

1. **Mechanism of Action – Description**  
   - The NeuroCortex Sync operates by amplifying the brain's natural neuroplasticity—the ability of neural networks in the brain to change through growth and reorganization. This device would work by detecting and mapping existing neural patterns in the user's brain through EEG (electroencephalogram) technology. Once these patterns are identified, NCS uses biofeedback and transcranial magnetic stimulation to align the user’s brainwaves with a preloaded set of patterns associated with the target knowledge. This process enhances synaptic plasticity, facilitating rapid encoding of new information, akin to installing software onto a computer instantly. The dynamic interaction involves the modulation of neural excitability and neurotransmitter release to replicate a learning process instantaneously without the conventional timeframes.

2. **Mechanism of Action – Key Questions**  
   - How can we accurately map

In [253]:
# I have elements. I have dimesions. tell the billion stories. it either works or it doesn't.
# tell a complete story of this element along this dimension. 
# expand elements, expand dimensions. 
# tell a complete story along that dimensions

# data checks the accuracy/completeness of a story. data checks the accuracy/completeness of a 

In [299]:
# tell a complete story along each dimension
elements = process_response(dimensions_resp, sys_msg=SYS_PROMPT_PARSE_ELEMENTS_MODIFIED)
print(elements)

[{'NeuroCortex Sync (NCS)': "1. **Mechanism of Action – Description**  \n   - The NeuroCortex Sync operates by amplifying the brain's natural neuroplasticity—the ability of neural networks in the brain to change through growth and reorganization. This device would work by detecting and mapping existing neural patterns in the user's brain through EEG (electroencephalogram) technology. Once these patterns are identified, NCS uses biofeedback and transcranial magnetic stimulation to align the user’s brainwaves with a preloaded set of patterns associated with the target knowledge. This process enhances synaptic plasticity, facilitating rapid encoding of new information, akin to installing software onto a computer instantly. The dynamic interaction involves the modulation of neural excitability and neurotransmitter release to replicate a learning process instantaneously without the conventional timeframes.\n\n2. **Mechanism of Action – Key Questions**  \n   - How can we accurately map knowl

In [300]:
element_dimensions = {}
for element in elements:
    n, t = list(element.items())[0]
    elem_dims = process_response(t)
    dims_lst = []
    for dim in elem_dims:
        _, v = list(dim.items())[0]
        dims_lst.append(v)
    element_dimensions[n] = dims_lst




    


In [305]:
elem_dims = list(element_dimensions.values())[0]
elem_name = list(element_dimensions.keys())[0]
s = ""
for d in elem_dims:
    s += f"{d}\n\n"

print(s)


    



- The NeuroCortex Sync operates by amplifying the brain's natural neuroplasticity—the ability of neural networks in the brain to change through growth and reorganization. This device would work by detecting and mapping existing neural patterns in the user's brain through EEG (electroencephalogram) technology. Once these patterns are identified, NCS uses biofeedback and transcranial magnetic stimulation to align the user’s brainwaves with a preloaded set of patterns associated with the target knowledge. This process enhances synaptic plasticity, facilitating rapid encoding of new information, akin to installing software onto a computer instantly. The dynamic interaction involves the modulation of neural excitability and neurotransmitter release to replicate a learning process instantaneously without the conventional timeframes.

- How can we accurately map knowledge and skills to specific patterns of brain activity?
- What safety mechanisms are in place to prevent neural overload or uni

In [312]:
SYSTEM_PROMPT_DIMENSION_ELEMENTS = """
You are a systems biologist and cognitive architect.

Given:
- A human fantasia
- A conceptual element that helps realize the fantasia
- A set of exploratory questions about the mechanism of action

These elements should reflect:
- Processes, modules, or sub-systems implied by the questions
- What needs to be understood to answer those questions
- The internal architecture or behavior of the system in operation

Your task is to extract the **underlying components of the natural or target system** being engaged by the mechanism—not the tool's modules, but the **functional, biological, informational, or cognitive elements** within the process itself.

These may include anatomical structures, physiological mechanisms, cognitive functions, signal types, material behaviors, or process-specific substrates.

Each should be named and briefly described in the context of how it participates in or enables the mechanism of action being questioned.

Do not describe device architecture. Focus on the system the device is acting upon.

"""
USER_PROMPT_DIMENSION_ELEMENTS = """

Fantasia: {fantasia}

Element: {element}

Dimensions: 
{dimensions_text}
"""


In [310]:
print(usr_msg)



Fantasia: I wish that I could learn any topic instantly.

Element: NeuroCortex Sync (NCS)

Dimensions: - The NeuroCortex Sync operates by amplifying the brain's natural neuroplasticity—the ability of neural networks in the brain to change through growth and reorganization. This device would work by detecting and mapping existing neural patterns in the user's brain through EEG (electroencephalogram) technology. Once these patterns are identified, NCS uses biofeedback and transcranial magnetic stimulation to align the user’s brainwaves with a preloaded set of patterns associated with the target knowledge. This process enhances synaptic plasticity, facilitating rapid encoding of new information, akin to installing software onto a computer instantly. The dynamic interaction involves the modulation of neural excitability and neurotransmitter release to replicate a learning process instantaneously without the conventional timeframes.

- How can we accurately map knowledge and skills to spe

In [313]:
sys_msg = SYSTEM_PROMPT_DIMENSION_ELEMENTS
usr_msg = USER_PROMPT_DIMENSION_ELEMENTS.format(fantasia=fantasia, element=elem_name, dimensions_text=s)
dims_elems = ask(sys_msg, usr_msg)
print(dims_elems)

To understand the interaction of the NeuroCortex Sync with the human brain system for instant learning, several key components and processes are involved:

1. **Neural Networks and Synaptic Plasticity:**
   - **Neural Networks:** Complex interconnected networks of neurons responsible for processing and transmitting information. In learning, these networks undergo changes in connectivity strengths.
   - **Synaptic Plasticity:** The ability of synapses (the connection points between neurons) to strengthen or weaken over time, based on activity levels. This property is essential for learning and memory formation.

2. **EEG-Detected Neural Patterns:**
   - **Brainwave Patterns:** Oscillations detected through EEG, indicative of different states of consciousness and cognitive activities. Mapping these to learning signifies the brain's activity states associated with various knowledge tasks.
   - **Cortical Activity Mapping:** Using EEG to detect regional brain activity patterns related to s